In [57]:
from ML import *

In [9]:
df = pd.read_csv('datasets/forestfires.csv')
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [16]:
df.shape

(517, 13)

In [26]:
df.area.min()

0.0

***Encodage des variables non numériques:***

In [27]:
le = LabelEncoder()
le.fit(df.month)

LabelEncoder()

In [31]:
df.month = le.transform(df.month)

In [33]:
df.day = le.fit_transform(df.day)

In [36]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


***Extraction des features et du target:***

In [37]:
X_data = df.iloc[:,:-1]
Y_data = df.iloc[:,-1]

***Normalisation des données:***

In [49]:
temp_arr = df.loc[:5,'temp'].values
mean = df.temp.mean()
mu = df.temp.std()
temp_arr_norm = (temp_arr - mean)/mu
temp_arr_norm

array([-1.84085723, -0.15312996, -0.73866799, -1.82363553, -1.28976261,
        0.57018174])

In [51]:
scaler = StandardScaler()
scaler.fit(X_data)

StandardScaler()

In [54]:
X_data = scaler.fit_transform(X_data)

***Partitionnement du dataset en Train & Test :***

In [59]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_data,Y_data,test_size=.2,random_state=True)

In [60]:
X_train.shape, X_test.shape

((413, 12), (104, 12))

***Création du modèle de régression:***

* Regression linéaire multiple:

In [61]:
reg = LinearRegression()
reg.fit(X_train,y_train)
tr_pred = reg.predict(X_train)

In [62]:
print('MSE = ',mean_squared_error(y_train,tr_pred))
print('R2 = ',r2_score(y_train,tr_pred))

MSE =  3523.6956950408353
R2 =  0.01847711886887693


***Utilisation des algorithmes de régularisation:***

On va utiliser la méthode de <b>GridSearchCV</b> pour choisir les bons paramètres pour notre modèle

In [73]:
param = {'alpha':np.arange(20,step=.1)}
models = [Ridge(),Lasso(),ElasticNet()]

In [83]:
results = []
for model in models:
    grid = GridSearchCV(estimator=model,param_grid=param,cv=5,scoring='r2')
    grid.fit(X_train,y_train)
    results.append(grid)

C:\Program Files\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Program Files\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Program Files\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.471e+05, tolerance: 2.989e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/

sklearn.model_selection._search.GridSearchCV

In [84]:
print('Model       |  best score  |  best parameter')
print('-----------------------------------------')
for res,mod in zip(results,models):
    print(f'{mod}   | {res.best_score_} | {res.best_params_}')

Model       |  best score  |  best parameter
-----------------------------------------
Ridge()   | -0.10677518244021851 | {'alpha': 19.900000000000002}
Lasso()   | -0.028620135247265654 | {'alpha': 5.9}
ElasticNet()   | -0.026805726842262102 | {'alpha': 5.800000000000001}
